In [54]:
import os
import pandas as pd 
import math
import os
import numpy as np

If bed_start and bed_end are -1 that means that the alignment does not map to the interval. Another way you can come to this conclusion is by calculating the fraction of the read that overlaps and setting some cutoffs for what you consider VNTR, junction and Non-VNTR. 

In [68]:
# SETTING case specific variables 
case = 1
seq_len = 10000
vntr_len = 65
coverage = 30

# SETTING project specific variables 
proj_dir = '/frazer01/home/joreyna/repos/CSE-280a/Project/'
sample = 'sequence_{}_case_{}'.format(seq_len, case)
coverage_text = 'coverage_{}'.format(coverage)
out_dir = os.path.join(proj_dir, 'output/pipeline/sample/',  sample + '/', coverage_text + '/')

# WOA file
wao_fn = os.path.join(out_dir, sample + '.wao')   

# PARSE the bedtools intersect -wao result 
data = pd.read_table(wao_fn, header=None)
data.columns = ['Map_Template', 'Map_Start', 'Map_End', 'Read_Name', \
                '4', 'Read_Orientation', '6', '7', '8', '9', \
                'Map_BPs_Aligned', '11', 'Bed_Template', 'Bed_Start', 'Bed_End', 'Overlap']
data = data[['Map_Template', 'Map_Start', 'Map_End', 'Read_Name', \
                'Read_Orientation', 'Map_BPs_Aligned', 'Bed_Template', \
                 'Bed_Start', 'Bed_End', 'Overlap']]
data['Map_Start'] = pd.to_numeric(data['Map_Start'])
data['Map_End'] = pd.to_numeric(data['Map_End'])
data['Map_BPs_Aligned'] = [int(x.replace(',', '')) for x in data['Map_BPs_Aligned']]
data['Bed_Start'] = pd.to_numeric(data['Bed_Start'])
data['Bed_End'] = pd.to_numeric(data['Bed_End'])
data['Overlap'] = pd.to_numeric(data['Overlap'])

# FILTER out reads whose mapping length is not 100 bp's 
data = data[(data['Map_BPs_Aligned'] <= 150) & (data['Map_BPs_Aligned'] >= 50)] 

# ADD a copy number column from the template sequence 
data['Map_Copy_Number'] = data['Map_Template'].str.extract('seq([0-9]*)', expand=True)
data = data[['Map_Template',
 'Map_Copy_Number',
 'Map_Start',
 'Map_End',
 'Read_Name',
 'Read_Orientation',
 'Map_BPs_Aligned',
 'Bed_Template',
 'Bed_Start',
 'Bed_End',
 'Overlap',
 ]]
data = data[data['Overlap'] > 0]

In [74]:
data.head(10)

,Map_Template,Map_Copy_Number,Map_Start,Map_End,Read_Name,Read_Orientation,Map_BPs_Aligned,Bed_Template,Bed_Start,Bed_End,Overlap,Read_Overlap_Fraction,VNTR_Overlap_Fraction,Mapping_Status
2035,seq1,1,4983,5133,seq1-1955,-,150,seq1,5000,5065,65,0.433333,1.000000,Spanning
2131,seq3,3,5170,5320,seq1-1859,+,150,seq3,5000,5195,25,0.166667,0.128205,Non-VNTR
2133,seq1,1,4984,5134,seq1-1857,+,150,seq1,5000,5065,65,0.433333,1.000000,Spanning
2180,seq1,1,4875,5025,seq1-1810,+,150,seq1,5000,5065,25,0.166667,0.384615,Non-VNTR
2183,seq3,3,4891,5041,seq1-1807,-,150,seq3,5000,5195,41,0.273333,0.210256,Junction
2195,seq1,1,4992,5142,seq1-1795,-,150,seq1,5000,5065,65,0.433333,1.000000,Spanning
2368,seq3,3,4890,5040,seq1-1622,-,150,seq3,5000,5195,40,0.266667,0.205128,Junction
2390,seq1,1,5004,5154,seq1-1600,-,150,seq1,5000,5065,61,0.406667,0.938462,Junction
2437,seq1,1,4979,5129,seq1-1553,-,150,seq1,5000,5065,65,0.433333,1.000000,Spanning
2466,seq3,3,4861,5011,seq1-1524,+,150,seq3,5000,5195,11,0.073333,0.056410,Non-VNTR


## Location of reads on the template sequences

In [71]:
mapping_status = []
read_overlap_frac = []
vntr_overlap_frac = []
for index, sr in data.iterrows():
    if sr['Bed_End'] == -1:
        read_overlap_fraction = np.nan
        vntr_overlap_fraction = np.nan
    else:
        read_overlap_fraction = float(sr['Overlap']) / (sr['Map_End'] - sr['Map_Start'])
        vntr_overlap_fraction = float(sr['Overlap']) / (sr['Bed_End'] - sr['Bed_Start'])
        
    read_overlap_frac.append(read_overlap_fraction)
    vntr_overlap_frac.append(vntr_overlap_fraction)
    
    # Read completely spans the VNTR which occurs for copy number 1 and 2 
    if sr['Bed_Start'] > sr['Map_Start'] and sr['Bed_End'] < sr['Map_End']: 
        mapping_status.append('Spanning')
        
    # Reads are completely in the VNTR
    elif read_overlap_fraction == 1:
        mapping_status.append('VNTR')
        
    # Reads are partially in the VNTR
    elif read_overlap_fraction >= 0.20 and read_overlap_fraction < 1: 
        mapping_status.append('Junction')
        
    # Reads are completely outside of the VNTR   
    elif read_overlap_fraction > 0 and read_overlap_fraction < 0.20: 
        mapping_status.append('Non-VNTR')
        
    # Hard call. We'll call it grey to filter out. 
    else:
        mapping_status.append('Grey')
        
data['Read_Overlap_Fraction'] = read_overlap_frac
data['VNTR_Overlap_Fraction'] = vntr_overlap_frac
data['Mapping_Status'] = mapping_status
data = data[data['Mapping_Status'] != 'Grey'] # FILTER out grey regions 

In [72]:
data.head(10)

,Map_Template,Map_Copy_Number,Map_Start,Map_End,Read_Name,Read_Orientation,Map_BPs_Aligned,Bed_Template,Bed_Start,Bed_End,Overlap,Read_Overlap_Fraction,VNTR_Overlap_Fraction,Mapping_Status
2035,seq1,1,4983,5133,seq1-1955,-,150,seq1,5000,5065,65,0.433333,1.000000,Spanning
2131,seq3,3,5170,5320,seq1-1859,+,150,seq3,5000,5195,25,0.166667,0.128205,Non-VNTR
2133,seq1,1,4984,5134,seq1-1857,+,150,seq1,5000,5065,65,0.433333,1.000000,Spanning
2180,seq1,1,4875,5025,seq1-1810,+,150,seq1,5000,5065,25,0.166667,0.384615,Non-VNTR
2183,seq3,3,4891,5041,seq1-1807,-,150,seq3,5000,5195,41,0.273333,0.210256,Junction
2195,seq1,1,4992,5142,seq1-1795,-,150,seq1,5000,5065,65,0.433333,1.000000,Spanning
2368,seq3,3,4890,5040,seq1-1622,-,150,seq3,5000,5195,40,0.266667,0.205128,Junction
2390,seq1,1,5004,5154,seq1-1600,-,150,seq1,5000,5065,61,0.406667,0.938462,Junction
2437,seq1,1,4979,5129,seq1-1553,-,150,seq1,5000,5065,65,0.433333,1.000000,Spanning
2466,seq3,3,4861,5011,seq1-1524,+,150,seq3,5000,5195,11,0.073333,0.056410,Non-VNTR


## Old Algorithm for determining copy number (incomplete)

In [75]:
map_grp = data.groupby('Mapping_Status')
mapping_statuses = map_grp.groups.keys()
total_spanning_templates = len(map_grp.get_group('Spanning')['Map_Copy_Number'].unique())
copy_number = '?'
if len(mapping_statuses) == 1 and 'Non-VNTR' in mapping_statuses:
    copy_number = 0
elif len(mapping_statuses) == 2 and 'Junction' in mapping_statuses:
    copy_number = 1
elif total_spanning_templates == 2:
    copy_number = 2

In [76]:
copy_number

2

In [77]:
mapping_statuses

['Junction', 'Spanning', 'Non-VNTR', 'VNTR']

In [78]:
spanning = map_grp.get_group('Spanning').sort_values('Map_Copy_Number')

In [79]:
vntr = map_grp.get_group('VNTR').sort_values('Map_Copy_Number')
vntr_mt_grp = vntr.groupby('Map_Copy_Number')
vntr_mt_count = vntr_mt_grp.count()['Map_Template'].to_frame()
vntr_mt_count.columns = ['Counts']
#vntr_mt_count.columns = ['Counts']
vntr_mt_count['Read_Per_Copy'] = [sr['Counts']/int(index) for index, sr in vntr_mt_count.iterrows() ]
vntr_mt_count['Counts_Norm_by_Length'] = \
    [float(sr['Counts'])/(int(index)  * vntr_len) for index , sr in vntr_mt_count.iterrows() ]

In [80]:
vntr_mt_count

,Counts,Read_Per_Copy,Counts_Norm_by_Length
Map_Copy_Number,,,
3,12,4,0.061538
